In [1]:
#

In [27]:
# https://www.kaggle.com/c/santander-customer-satisfaction/overview
# https://www.kaggle.com/kobakhit/0-84-score-with-36-features-only
# https://vishalmnemonic.github.io/DC16/

In [3]:
#

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

/data/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/data/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [6]:
import xgboost as xgb

In [7]:
#

In [8]:
train = pd.read_csv("./santander_train.csv")

In [9]:
test = pd.read_csv("./santander_test.csv")

In [10]:
#

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76020 entries, 0 to 76019
Columns: 371 entries, ID to TARGET
dtypes: float64(111), int64(260)
memory usage: 215.2 MB


In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75818 entries, 0 to 75817
Columns: 370 entries, ID to var38
dtypes: float64(110), int64(260)
memory usage: 214.0 MB


In [13]:
#

In [14]:
remove = []
for col in train.columns:
    if train[col].std() == 0:
        remove.append(col)

In [15]:
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

In [16]:
remove = []
cols = train.columns
for i in range(len(cols)-1):
    v = train[cols[i]].values
    for j in range(i+1,len(cols)):
        if np.array_equal(v,train[cols[j]].values):
            remove.append(cols[j])

In [17]:
train.drop(remove, axis=1, inplace=True)
test.drop(remove, axis=1, inplace=True)

In [18]:
test_id = test.ID
test = test.drop(["ID"],axis=1)

In [20]:
train_sample = train.sample(frac=0.1)


In [22]:
X = train_sample.drop(["TARGET","ID"],axis=1)
y = train_sample.TARGET.values

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1729)

In [24]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6081, 306) (1521, 306) (6081,) (1521,)


In [25]:
print(test.shape)

(75818, 306)


In [26]:
#

In [28]:
xgb_model = xgb.XGBClassifier(max_depth=4, min_child_weight = 1, n_estimators=200, learning_rate=0.0202, gamma=0, nthread=4, subsample=0.6815, colsample_bytree=0.701, seed=1)


In [29]:
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.701,
       gamma=0, learning_rate=0.0202, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=200, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=1, silent=True, subsample=0.6815)

In [30]:
preds = xgb_model.predict(X_test)

/data/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

In [34]:
print(accuracy)

0.9658119658119658
